# The merge function


In [1]:
import pandas as pd

left_df = pd.DataFrame({
                    'firm': ['Accenture','Citi','GS'],
                    'varA': ['A1', 'A2', 'A3']})

right_df = pd.DataFrame({
                    'firm': ['GS','Chase','WF'],
                    'varB': ['B1', 'B2', 'B3'],
                    'varc': ['C1', 'C2', 'C3']})    


In [2]:
ChosenOne = pd.DataFrame({
                    'tic': ['GS','GS','GS','F',],
                    'varB': ['B1', 'B2', 'B3','B4'],
                    'varc': ['C1', 'C2', 'C3','C4']})    

CurryForThree = pd.DataFrame({
                    'var1': ['GS','GS','GS'],
                    'varD': ['D1', 'D2', 'D3'],
                    'varE': ['E1', 'E2', 'E3']}).set_index('var1')

Let use shift+tab to talk about the parameters.

Put your cursor in the merge function below and hit shift tab

In [3]:
left_df

,firm,varA
0,Accenture,A1
1,Citi,A2
2,GS,A3


In [4]:
right_df

,firm,varB,varc
0,GS,B1,C1
1,Chase,B2,C2
2,WF,B3,C3


In [5]:
ChosenOne

,tic,varB,varc
0,GS,B1,C1
1,GS,B2,C2
2,GS,B3,C3
3,F,B4,C4


In [6]:
CurryForThree

,varD,varE
var1,,
GS,D1,E1
GS,D2,E2
GS,D3,E3


In [7]:
left_df.merge(right_df)

,firm,varA,varB,varc
0,GS,A3,B1,C1


The main parameters:
- right: the data we merge to
- how: which set of keys do we have after the merge
- on and its variants 
    - on: the variable or variables to merge with
    - names must be in both dataset the same
    - if the key vars in datasets have diff names, **my preference is to rename so they match (ensures no empty rows of the key vars after merge)** (insread of using left_on/right_on)
- indicator: always use - useful for diagnostics! (`value_counts()`)
- validate: always use - checks your conception about merge
    - always use the most restrictive version
    - never use "m:m"


## Part 1

_Prof: Leave the "how" slide on the board_

Some work with the mechanics:

- Q0 Merge both datasets above with each possible value of `how`. How many observations result from each of the four merges?

In [8]:
left_df.merge(right_df, how='inner')

# inner - gives back a DF with only the "keys" in both dfs
# vocab: "key" in the merge, is the variables you merge on

# what variarblae are we merging on?
# if you dont tell it: any and all variables in both datasets 

# always specify which vars

,firm,varA,varB,varc
0,GS,A3,B1,C1


In [9]:
left_df.merge(right_df, on='firm', how='outer')

# outer gives back all combos of keys

# if a key isn't in both, any variables for that key are filled with a missing value
# NaN = not a number      (np.nan, None)

,firm,varA,varB,varc
0,Accenture,A1,NaN,NaN
1,Citi,A2,NaN,NaN
2,GS,A3,B1,C1
3,Chase,NaN,B2,C2
4,WF,NaN,B3,C3


In [10]:
left_df.merge(right_df, on='firm', how='left')

# probably the most common usage of merge
# like ADDNIG A variable to the dataset you're analyzing 

# returns the "left dataset" + vars from right dataset 
# caveat: see 1:M discussion later (left merges can add rows if 1:m)

,firm,varA,varB,varc
0,Accenture,A1,NaN,NaN
1,Citi,A2,NaN,NaN
2,GS,A3,B1,C1


In [11]:
left_df.merge(right_df, on='firm', how='right') 

,firm,varA,varB,varc
0,GS,A3,B1,C1
1,Chase,NaN,B2,C2
2,WF,NaN,B3,C3


In [12]:
# cross - if you want to use it, ask me first!

- Q1 Compare `left_df.merge(right_df)` and `pd.merge(left_df, right_df)`. Are they the same or different? What do we learn from this?

They are the same!

`left_df.merge(right_df)` can be be used in "chains".

In [13]:
left_df.merge(right_df)

,firm,varA,varB,varc
0,GS,A3,B1,C1


In [14]:
pd.merge(left_df, right_df)

,firm,varA,varB,varc
0,GS,A3,B1,C1


## On parameter

- a variable name or a list of variable names to merge on
- YOU SHOULD EXPLCITLY INCLUDE THIS IN EVERY SINGLE MERGE 
    - sometimes you'll use "alternatives" to "on"

In [15]:
left_df.merge(right_df,on=['firm'])

,firm,varA,varB,varc
0,GS,A3,B1,C1


- Q2 Successfully do an outer merge between `left_df` and `ChosenOne`. Then try to do an outer merge between `left_df` and `CurryForThree` (both are defined below)

In [16]:
# when the variables have diff names... left_on/right_on
# if the keys are the index variables... left_index/right_index 
left_df.merge(ChosenOne,
              how = 'outer',
              left_on = 'firm',
              right_on = 'tic'
             )

,firm,varA,tic,varB,varc
0,Accenture,A1,NaN,NaN,NaN
1,Citi,A2,NaN,NaN,NaN
2,GS,A3,GS,B1,C1
3,GS,A3,GS,B2,C2
4,GS,A3,GS,B3,C3
5,NaN,NaN,F,B4,C4


## Uh-oh! 

Notice how the firm AND ticker variables are bad now? Neither are complete... missing values!

The alternative below (rename one df's columns) is better: ends up with 1 variable for the firm/tic, no missing values!



In [17]:
# or just rename the variables and use on
# notice: this rename is tempoerary, for the merge only!
left_df.merge(ChosenOne.rename(columns={'tic':'firm'}),
              how = 'outer',
              on = 'firm',
             )

,firm,varA,varB,varc
0,Accenture,A1,NaN,NaN
1,Citi,A2,NaN,NaN
2,GS,A3,B1,C1
3,GS,A3,B2,C2
4,GS,A3,B3,C3
5,F,NaN,B4,C4


In [18]:
# exercise: left_df + CurryForThree

- Q3 Do an outer merge with `left_df` and `right_df` and output the source of each observation by using the "indicate" option.

In [19]:
# use good names for merged datasets (never just "merged")
# I try to name df's with the observation level

firm_df = left_df.merge(right_df,
                        how='outer',
                        on='firm',
                        indicator=True)

# GREAT HABIT: check the source of observations... what you expected?
print(firm_df['_merge'].value_counts())

# another good habit: print the length of the df before and after merge
print('before',len(left_df),'after',len(firm_df))


left_only     2
right_only    2
both          1
Name: _merge, dtype: int64
before 3 after 5


- Q4 Repeat the outer merge we just did, but four times: try each possible value of "validate"

In [20]:
left_df.merge(right_df,validate='1:1') # it works
left_df.merge(right_df,validate='m:1') # it works
left_df.merge(right_df,validate='1:m') # it works
left_df.merge(right_df,validate='m:m') # it works

# all of these work because 
# 1:1 is a "special case" of m:1, 1:m, m:m

# implication: m:m is "stupid'... it always works, no errors thrown
# you want to use the "most restrictive" option you think is right 

,firm,varA,varB,varc
0,GS,A3,B1,C1


Now, let's add two more NBA themed datasets to the party: 

## Part 2

- Q5: [Guess what category of join](https://ledatascifi.github.io/ledatascifi-2023/content/03/05b_merging.html#categories-of-joins) each of the following merges are?  
    1. `left_df` and `right_df` (you already know from Q4)
    1. `left_df` and `ChosenOne`
    1. `CurryForThree` and `left_df`
    1. `ChosenOne` and `CurryForThree`

In [21]:
# review this... take a guess

# use the merge_type fuction

- Q6: Do each of those four merge with `how='inner'` as an option. What is the length of each resulting dataframe?

- Q7: Do an outer merge of `left_df` and `ChosenOne`. How many observations are in the resulting data, and why is it different than we foudn in Q6?

- Q8: Merge these next two datasets with `how='inner'` as an option. What s the length of the resulting dataframe and do you think it's right?

In [22]:
poppop = pd.DataFrame({
                    'tic': ['TSLA','TSLA','GM'],
                    'varB': ['2016', '2017', '2018'],
                    'varc': ['C1', 'C2', 'C3']})    

CurryForThree = pd.DataFrame({
                    'var1': ['F','TSLA','TSLA'],
                    'varD': ['2016', '2017', '2018'],
                    'varE': ['E1', 'E2', 'E3']}).set_index('var1')

## Part 3 - Collecting tips

You should, after class, collect tips about
- The most common use of merging and safeguards you can use
- When should you create variables - before or after a merge?
- Best practices for merging

